In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 25 16:44:52 2023

@author: cyh
"""

# function：
#     1. load pointcloud data
#     2. filter the ground pointcloud
#     3. do clustering

# # If run in vscode, change the path to site-packages in anaconda env
# import sys
# sys.path.append('/home/cyh/anaconda3/lib/python3.8/site-packages')

# import
import numpy as np
import open3d as o3d
import struct
from sklearn import cluster, datasets, mixture
import matplotlib.pyplot as plt
from pandas import DataFrame
from pyntcloud import PyntCloud
import math
import random
from collections import defaultdict
from sklearn.cluster import DBSCAN

import json


# visualize using matplotlib
def Point_Cloud_Show(points):
    fig = plt.figure(dpi=150)
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], cmap='spectral', s=2, linewidths=0, alpha=1, marker=".")
    plt.title('Point Cloud')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    plt.show()

# read pointcloud data from file
# Input:
#     path: file path
# Output:
#     pointcloud array
def read_velodyne_bin(path):
    '''
    :param path:
    :return: homography matrix of the point cloud, N*3
    '''
    file_data = np.fromfile(path, dtype=np.float32)
    points = file_data.reshape((-1, 5))[:, :3]
    return points

# filter the ground pointcloud
# Input:
#     data: one frame of pointcloud
# Output:
#     segmengted_cloud: pointcloud after filtering
def ground_segmentation(data):
    # initialize data
    idx_segmented = []
    segmented_cloud = []
    iters = 100     # maximum iteration  
    sigma = 0.4     # maximum accepted error between data and model 数据和模型之间可接受的最大差值  
    ## plane model:  aX + bY + cZ +D= 0
    best_a = 0
    best_b = 0
    best_c = 0
    best_d = 0
    pretotal = 0    # number of inliers in last iteration
    P = 0.99        # the probability of getting the correct model 希望的到正确模型的概率
    n = len(data)   # number of points 
    outline_ratio = 0.6   #e: outline_ratio  
    for i in range(iters):
        ground_cloud = []
        idx_ground = []
        # step1 choose the smallest datset to estimate the model, 3 points for a plane
        # 选择可以估计出模型的最小数据集，对于平面拟合来说，就是三个点
        sample_index = random.sample(range(n),3)    # randomly select 3 points from dataset
        point1 = data[sample_index[0]]
        point2 = data[sample_index[1]]
        point3 = data[sample_index[2]]
        # step2 solve the model
        ## first solve the normal vector 先求解法向量
        point1_2 = (point1-point2)      # vector poin1 -> point2
        point1_3 = (point1-point3)      # vector poin1 -> point3
        N = np.cross(point1_3,point1_2)            # get the normal vector of plane 向量叉乘求解 平面法向量
        ## slove model parameter a,b,c,d
        a = N[0]
        b = N[1]
        c = N[2]
        d = -N.dot(point1)
        # step3 use all the data to count the number of inliers 
        # 将所有数据带入模型，计算出“内点”的数目；(累加在一定误差范围内的适合当前迭代推出模型的数据)
        total_inlier = 0
        pointn_1 = (data - point1)    # sample（三点）外的点 与 sample内的三点其中一点 所构成的向量
        distance = abs(pointn_1.dot(N))/ np.linalg.norm(N)     # distance
        # judge the inliers by distance
        idx_ground = (distance <= sigma)
        total_inlier = np.sum(idx_ground == True)    # number of inliers
        # compare the performance of model 
        if total_inlier > pretotal:                                           #     log(1 - p)
            iters = math.log(1 - P) / math.log(1 - pow(total_inlier / n, 3))  #N = ------------
            pretotal = total_inlier                                               #log(1-[(1-e)**s])
            # the best model param
            best_a = a
            best_b = b
            best_c = c
            best_d = d

        # enough inliers?
        if total_inlier > n*(1-outline_ratio):
            break
    print("iters = %f" %iters)
    # points after segmentation
    idx_segmented = np.logical_not(idx_ground)
    ground_cloud = data[idx_ground]
    segmented_cloud = data[idx_segmented]
    return ground_cloud,segmented_cloud


# do clustering
# Input:
#     data: cloud points without ground cloud
# Output:
#     clusters_index： class index of every points
def dbscan_clustering(data):
    # sklearn dbscan
    # eps: scan radius;  min_samples: min samples in the circle; n_jobs ：CPU form
    cluster_index = DBSCAN(eps=1,min_samples=3,n_jobs=-1).fit_predict(data)
    print(cluster_index)
    return cluster_index



# visualize clustering results with different color
def plot_clusters(segmented_ground, segmented_cloud, cluster_index, bbox):
    
    # Visualize segmentation results using Open3D
    def colormap(c, num_clusters):
        # Colormap for segmentation result
        # outlier:
        if c == -1:
            color = [1]*3
        # surrouding object:
        else:
            color = [0] * 3
            color[c % 3] = c/num_clusters

        return color
    
    # list for plot
    plot_list = []

    # ground element:
    pcd_ground = o3d.geometry.PointCloud()
    pcd_ground.points = o3d.utility.Vector3dVector(segmented_ground)
    pcd_ground.colors = o3d.utility.Vector3dVector(
        [
            [0, 0, 255] for i in range(segmented_ground.shape[0])
        ]
    )
    plot_list.append(pcd_ground)

    # surrounding object elements:
    pcd_objects = o3d.geometry.PointCloud()
    pcd_objects.points = o3d.utility.Vector3dVector(segmented_cloud)
    num_clusters = max(cluster_index) + 1
    pcd_objects.colors = o3d.utility.Vector3dVector(
        [
            colormap(c, num_clusters) for c in cluster_index
        ]
    )
    plot_list.append(pcd_objects)
    
    # bounding box
    # bbox: [position–x, position–y , position–z , size-x, size-y, size-z, class]
    for i in range(len(bbox)):
        bounding_box = o3d.geometry.AxisAlignedBoundingBox()
        
        min_bound = [bbox[i][0] - bbox[i][3] / 2, bbox[i][1] - bbox[i][4] / 2, bbox[i][2] - bbox[i][5] / 2]
        max_bound = [bbox[i][0] + bbox[i][3] / 2, bbox[i][1] + bbox[i][4] / 2, bbox[i][2] + bbox[i][5] / 2]
        
        bounding_box.min_bound = min_bound
        bounding_box.max_bound = max_bound
        bounding_box.color = colormap(bbox[i][6], num_clusters)
        
        plot_list.append(bounding_box)
        
    

    # visualize
    o3d.visualization.draw_geometries(plot_list)
    # o3d.visualization.draw_geometries([pcd_ground, pcd_objects, bounding_box, bounding_box2])

    

def main():
    iteration_num = 10    # num of files
    
    res = {}

    for i in range(iteration_num):
        # filename = '/home/cyh/文档/nus/semester2/ME5413/HW1/me5413_homework1/me5413/1_lidar/lidar_data/frame' + \
        #         str(i+1) + '.pcd.bin'
        filename = './lidar_data/frame' + str(i+1) + '.pcd.bin'
        print('clustering pointcloud file:', filename)
        
        
        origin_points = read_velodyne_bin(filename)     # read data
        origin_points_df = DataFrame(origin_points,columns=['x', 'y', 'z'])  # [0,3)
        point_cloud_pynt = PyntCloud(origin_points_df)  # store in structs
        point_cloud_o3d = point_cloud_pynt.to_instance("open3d", mesh=False) # to instance
    
        # show initial cloudpoints
        # o3d.visualization.draw_geometries([point_cloud_o3d])
    
        # ground segmentation
        ground_points, segmented_points = ground_segmentation(data=origin_points)
        
        # show clustering results
        cluster_index = dbscan_clustering(segmented_points)
        # print(max(cluster_index) + 1)
        
        
        # position–x, position–y , position–z , size-x, size-y, size-z, class, confidence
        bbox = []
        
        for j in range(max(cluster_index) + 1):
            
            # points of class j
            points_inClass = np.argwhere(cluster_index == j)
            
            # num of points in class j
            num = len(points_inClass)
            
            # xyz coordinates of points in class j
            points_xyz = segmented_points[points_inClass].reshape(num, -1)
            
            # position–x, position–y , position–z of bbox
            bbox_px = sum(points_xyz[:,0]) / num
            bbox_py = sum(points_xyz[:,1]) / num
            bbox_pz = sum(points_xyz[:,2]) / num
            
            # size-x, size-y, size-z of bbox
            bbox_sx = max(points_xyz[:,0])-min(points_xyz[:,0])
            bbox_sy = max(points_xyz[:,1])-min(points_xyz[:,1])
            bbox_sz = max(points_xyz[:,2])-min(points_xyz[:,2])
            
            bbox.append([bbox_px, bbox_py, bbox_pz, bbox_sx, bbox_sy, bbox_sz, j])
        
        # visualization
        plot_clusters(ground_points, segmented_points, cluster_index, bbox)
        

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(cluster_index)) - (1 if -1 in cluster_index else 0)
        n_noise_ = list(cluster_index).count(-1)
        
        bbox = np.array(bbox).tolist()
        
        res['Lidar clustering results of frame' + str(i+1)] =       \
            {'Number of ground points': len(ground_points),         \
             'Number of surrounding points': len(segmented_points), \
             'Estimated number of clusters': n_clusters_,           \
             'Estimated number of noise points': n_noise_,          \
             'List of bounding box objects':bbox
             }


    new_dict = json.dumps(res, indent=1)
    with open("./lidar_clustering.json","w", newline='\n') as f:
        f.write(new_dict)
        print("Write successfully!")
        

if __name__ == '__main__':
    main()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
clustering pointcloud file: ./lidar_data/frame1.pcd.bin
iters = 38.293683
[0 1 2 ... 2 3 2]
clustering pointcloud file: ./lidar_data/frame2.pcd.bin
iters = 45.096749
[0 0 1 ... 0 0 0]
clustering pointcloud file: ./lidar_data/frame3.pcd.bin
iters = 71.018144
[0 0 0 ... 1 1 1]
clustering pointcloud file: ./lidar_data/frame4.pcd.bin
iters = 77.128383
[0 0 0 ... 0 0 2]
clustering pointcloud file: ./lidar_data/frame5.pcd.bin
iters = 57.940245
[0 0 0 ... 0 0 0]
clustering pointcloud file: ./lidar_data/frame6.pcd.bin
iters = 42.301543
[0 0 0 ... 1 0 1]
clustering pointcloud file: ./lidar_data/frame7.pcd.bin
iters = 45.186125
[0 0 0 ... 1 2 2]
clustering pointcloud file: ./lidar_data/frame8.pcd.bin
iters = 65.977430
[0 0 0 ... 1 0 2]
clustering pointcloud file: ./lidar_data/frame9.pcd.bin
iters = 47.778082
[0 0 0 ... 2 2 2]
clu